Misconception Data
===

In [93]:
import json
import os
import re
from collections import Counter
from pathlib import Path

import dotenv
import matplotlib.pyplot as plt
import numpy as np
import openai
import pandas as pd
import requests
import tiktoken

In [17]:
dotenv.load_dotenv("../.env")
openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
result = requests.get("https://raw.githubusercontent.com/creature-ai/math-misconceptions/main/misconceptions.json")

In [3]:
misconception_list = json.loads(result.text)
len(misconception_list)

58

In [4]:
df = pd.DataFrame(misconception_list)
df.columns

Index(['Misconception', 'ID', 'Topic', 'Example1',
       'Example1 Graph Description', 'Paper 1', 'Example2',
       'Example2 Graph Description', 'Paper 2', 'Example3',
       'Example3 Graph Description', 'Paper 3', 'Example4',
       'Example4 Graph Description', 'Paper 4'],
      dtype='object')

In [5]:
df

,Misconception,ID,Topic,Example1,Example1 Graph Description,Paper 1,Example2,Example2 Graph Description,Paper 2,Example3,Example3 Graph Description,Paper 3,Example4,Example4 Graph Description,Paper 4
0,when students write incorrectly transpose vari...,MaE46,"Variables, expressions, and operations",Q:\nWrite an algebraic expression to represent...,,"Capraro, M. M., & Joffrion, H. (2006)\np. 7",Q: \nWrite the expression that represents: “Ta...,,"Capraro, M. M., & Joffrion, H. 2006\np.18",Q: \nAt a meeting there are five more women th...,,"Stacey, K & MacGregor M (2014)\n\np. 5","Q: ""z is equal to the sum of 3 and y."" Write t...",,"Stacey, K & MacGregor M (2014)\n\np. 5"
1,when students incorrectly view variables as la...,MaE47,"Variables, expressions, and operations",Q:\nThe following question is about this expre...,,Asquith et al. 2007p. 11,Q:\nDavid is 10 cm taller than Con. Con is h c...,,"Stacey and MacGregor, 1997b\np. 2",Q:\nGiven the equation: a = 28 + b\nWhich of t...,,"Stacey and MacGregor, 1997b\np. 2",Q: \nSue weighs 1 kg less than Chris. Chris ...,,MacGregor and Stacey 1997\np. 6
2,when students believe that two variables in th...,MaE48,"Variables, expressions, and operations","Q: \nh + m + n = h + p + n always, sometimes,...",,Stephens (2005)\np. 1,Q:\nRicardo has 8 pet mice. He keeps them in t...,,Stephens (2005)\np. 2-4,Q:\nAre these number sentences true or false?\...,,Stephens (2005)\np. 1,0,,
3,when students fail to understand that variable...,MaE49,"Variables, expressions, and operations",Q:\nA store is having a sale. They have advert...,,"Bair, S. L., & Rich, B. S. 2011\np. 12",Q:\nWhat can you say about r if r = s + t and ...,,Kuchemann (1978)\np.3,Q: \nWhich is larger task: Can you tell which...,,Kuchemann 1978\np. 5,Q:\nWhat can you say about c if c + d = 10 and...,,Kuchemann (1978)\np.3
4,when students believe that the commutative and...,MaE50,"Variables, expressions, and operations",Q:\nAre these statements true or false? \n2 – ...,,Baroudi 2006\np. 3,Q:\nSimplify: 3(6x - 4) + 2(3x - 3)\nShow your...,,"Bush, 2011\np. 215-216","Q: \nWithout using the calculator, just by loo...",,"Linchevski, L., & Livneh, D. 1999\np. 12",Q: \nBlanca's aquarium has three types of fish...,,"Bush, 2001\np. 217-219"
5,when students hold a rigid understanding of pr...,MaE23,Ratios and proportional reasoning,"Q:\nAt ARCO, gas sells for $1.13 per gallon. T...",,"Kilpatrick, J., Swafford, J. O., & Findell, B....",Q: \nAdrian has conquered only 6 giants in his...,,"Brown, G., & Quinn, R. J. (2006) \np. 5","Q:\n Which of the following drinks, A or B, ...",,Lamon 2012\n(no link),Q: \nConsider two marigolds that were 8 inches...,,"Kilpatrick, J., Swafford, J. O., & Findell, B...."
6,when students show an inability to recognize t...,MaE24,Ratios and proportional reasoning,"Q:\nTo bake donuts, Mariah needs 8 cups of flo...",,"Singh, 2000\np. 17","Q:\nTo bake 6 cakes, you need 15 eggs. Using t...",,"Singh, 2000\np. 19","Q:\nIf 4 cents can buy 6 sweets, how many can ...",,"Singh, 2000\np. 8","Q: \nIn baking a cake, for every 8 eggs, one...",,"Sigh, 2000\np. 18"
7,when students have confusions with ratios writ...,MaE25,Ratios and proportional reasoning,"Q:\nExpress ""2/3 dollars per balloon"" with equ...",,"Kilpatrick, J., Swafford, J. O., & Findell, B....",Q:\n3:2 = 24:x\nFind the value of x\n\nA:\n32 ...,,"Kilpatrick, J., Swafford, J. O., & Findell, B....",Q:\n If 5/8 = x/24 then find x\n\nA:\nx = 5...,,"Brown, G., & Quinn, R. J. (2006) \np. 6",Q:\nGreg conducts a survey of 100 classmates t...,,Bush 2011\np. 188
8,when students present a lack of recognition of...,MaE26,Ratios and proportional reasoning,"Q:\nIf 5 quarters can buy 100 candies, how man...",,Ojose. B. 2015\np. 4,Q: \n(1/3)*a = ?\n\nA:\n1/3a\n[The learner did...,,"Brown, G., & Quinn, R. J. (2006) \np. 6",Q:\nReduce: (3 + 4)/2\n\nA:\n4 ÷ 2 + 3 = 5\n\...,,"Brown, G., & Quinn, R. J. (2006) \np. 7",Q:\nYou had a picture on your computer and you...,,Lamon 2012\n(no link)
9,when

In [8]:
df.ID.value_counts().head()

ID
         3
MaE47    1
MaE11    1
MaE12    1
MaE13    1
Name: count, dtype: int64

In [10]:
# not sure what's going on with the misconception without an ID
df[~df.ID.str.startswith("MaE")]

,Misconception,ID,Topic,Example1,Example1 Graph Description,Paper 1,Example2,Example2 Graph Description,Paper 2,Example3,Example3 Graph Description,Paper 3,Example4,Example4 Graph Description,Paper 4
16,when students lack understanding and don't see...,-,"Patterns, relationships, and functions",Q: Make a table of values that would produce t...,Q: Make a table of values that would produce t...,"Kalchman, M., & Koedinger, K. R. 2005\np. 3-8","Q:\nFor the function f(x) = 2x^2 + 3x - 1, (i)...","Q:\nFor the function f(x) = 2x^2 + 3x - 1, (i)...","Bair, S. L., & Rich, B. S. 2011\np. 14","Q:\nFor the function f(x) = 2x^2 + 3x - 1, (ii...","Q:\nFor the function f(x) = 2x^2 + 3x - 1, (ii...","Bair, S. L., & Rich, B. S. 2011 \np. 14",Q: \nAfter having solving the equation 2 = 4x ...,,"Kalchman, M., & Koedinger, K. R. (2005)\np. 20"
55,,,,0,,,3,,,,,,12,,
56,,,,1,,,1,,,,,,7,,
57,,,,"Q. Write an expression that represents: ""“Juli...",,,,,,,,,,,


### Prompting tests

In [18]:
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [70]:
mae_spreadsheet_string = ""
misconception_df = df[df.ID.str.startswith("MaE")]
mae_count = len(misconception_df)
for row in misconception_df.sample(frac=1).itertuples():
    s = f"ID: {row.ID}; Description: {row.Misconception}; Example 1: {row.Example1}; Example 2: {row.Example2}\n"
    # print(s)
    mae_spreadsheet_string += s
n_tokens = len(tokenizer.encode(mae_spreadsheet_string))
n_tokens

11686

In [33]:
data_dir = Path("../data")
assert data_dir.exists()

In [34]:
incorrect_answer_csv_filepath = data_dir / "derived" / "incorrect_answers.csv"
incorrect_lesson_df = pd.read_csv(incorrect_answer_csv_filepath)
print(incorrect_lesson_df.shape)
incorrect_lesson_df.sample(n=3)

(153, 3)


,unique_index,answer,incorrect_answer
114,1649,Even,Odd
97,1189,4,20
100,1192,6,40


In [35]:
lesson_df = pd.read_pickle(data_dir / "derived" / "rori_lessons.pkl")
lesson_df = lesson_df.reset_index().rename(columns={"index": "unique_index"})
lesson_df.shape

(2570, 21)

In [53]:
lesson_codes = set()
lesson_codes_with_images = set()
for lesson_code, lesson in lesson_df.groupby("lesson_code"):
    lesson_codes.add(lesson_code)
    has_image_explanation = ((lesson.is_explanation) & (lesson.has_image)).any()
    has_image_question = ((~lesson.is_explanation) & (lesson.has_image)).any()
    has_image = has_image_explanation or has_image_question
    if has_image:
        lesson_codes_with_images.add(lesson_code)
len(lesson_codes_with_images), len(lesson_codes)

(74, 225)

In [54]:
sdf = lesson_df[(~lesson_df.lesson_code.isin(lesson_codes_with_images)) & (lesson_df.grade.isin([6, 7, 8]))]
f"{len(sdf)} / {len(lesson_df)}", f"{sdf.lesson_code.nunique()} / {lesson_df.lesson_code.nunique()}"

('589 / 2570', '52 / 225')

In [55]:
# merge in incorrect lessons
sdf = sdf.merge(incorrect_lesson_df[["unique_index", "incorrect_answer"]], on="unique_index", how="left")
sdf.incorrect_answer.notna().sum()

28

In [57]:
sdf = sdf[sdf.incorrect_answer.notna()]
len(sdf)

28

In [58]:
sdf.lesson_code.value_counts()

lesson_code
G8.N3.2.4.1    7
G8.A2.1.1.1    6
G6.S2.1.1.1    5
G8.N4.2.3.1    4
G7.N5.1.1.2    1
G7.N5.1.1.1    1
G6.N1.1.2.1    1
G6.N2.1.2.3    1
G7.N4.2.1.1    1
G8.N3.2.4.2    1
Name: count, dtype: int64

In [63]:
sdf[["unique_index", "lesson_code", "question", "answer", "incorrect_answer"]]

,unique_index,lesson_code,question,answer,incorrect_answer
14,434,G8.A2.1.1.1,A car travels 120 miles in H hours. The expres...,A,B
15,435,G8.A2.1.1.1,"In stable, there are z horses. 10 of them are ...",D,A
16,436,G8.A2.1.1.1,John scores 3 more points than James p scores....,B,A
17,437,G8.A2.1.1.1,If you multiply four by n and decreased by six...,A,B
18,438,G8.A2.1.1.1,"The difference of thirteen and y, then add x. ...",C,D
21,441,G8.A2.1.1.1,There are x ballpens in a pack. \nI buy 4 pack...,B,A
25,445,G8.N3.2.4.1,8/10 = _____%,80,0.8
26,446,G8.N3.2.4.1,2/5 = _____%,40,10
27,447,G8.N3.2.4.1,6/20 = _____%,30,60
28,448,G8.N3.2.4.1,34/100 = _____%,34,340


In [64]:
exclude_indices = [2205, 2549, 2550, 2551, 2558]
sdf = sdf[~sdf.unique_index.isin(exclude_indices)]
len(sdf)

23

In [78]:
for row in sdf.itertuples():
    break
row.question

'A car travels 120 miles in H hours. The expression represents the average speed in miles per hour is ___\nSpeed = distance/time\nA. 120/H\nB. 120 x H\nC. 120 + H\nD. H/120\nChoose from the options A, B, C and D.'

In [90]:
model = "gpt-3.5-turbo-16k"
prompt = """I’ll give you a spreadsheet with a list of MaEs. 
Each MaE has an ID, a description of the MaE, and 2 examples of the MaE. 
Then I'll show you an incorrect answer to a math question. 
I want you to tell me how many of the {mae_count} MaEs you can identify in the answer, identifying as many as you can.

Spreadsheet:
{mae_spreadsheet_string}

Question: {question}
Answer: {answer}
Incorrect Answer: {incorrect_answer}

Which MaEs are in the incorrect answer?
"""

results = []
for row in sdf.itertuples():
    formatted_prompt = prompt.format(
        mae_spreadsheet_string=mae_spreadsheet_string,
        mae_count=mae_count,
        question=row.question,
        incorrect_answer=row.incorrect_answer,
        answer=row.answer,
    )
    completion = openai.ChatCompletion.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": formatted_prompt,
            }
        ],
    )
    result = completion["choices"][0]["message"]["content"]
    results.append(result)

In [91]:
m_df = sdf.copy()
m_df["gpt_result"] = results

In [100]:
def get_maes(result):
    return re.findall("MaE[0-9]+", result)


mae_lists = []
for result in m_df.gpt_result:
    mae_lists.append(get_maes(result))

In [102]:
m_df["n_maes"] = [len(mae_list) for mae_list in mae_lists]

In [105]:
m_df[m_df.n_maes == 0][["question", "answer", "incorrect_answer", "gpt_result", "n_maes"]]

,question,answer,incorrect_answer,gpt_result,n_maes
28,34/100 = _____%,34,340,One possible MaE in the incorrect answer is wh...,0
29,125/1000 = ____ %,12.5,125,There are no MaEs identified in the incorrect ...,0
37,2 x 2 = ___,2^2,4,"Based on the given spreadsheet, there are no i...",0
56,√225 = __,15,25,There are no MaEs present in the incorrect ans...,0
164,Two hundred fifty three thousand three hundred...,=,2,There are no MaEs in the incorrect answer.,0


In [112]:
c = Counter()
for mae_list in mae_lists:
    c.update(mae_list)
c.most_common(5)

[('MaE7', 5), ('MaE52', 4), ('MaE47', 3), ('MaE29', 3), ('MaE15', 2)]

In [115]:
m_df[["lesson_code", "grade", "question", "answer", "incorrect_answer", "gpt_result", "n_maes"]].to_csv(
    data_dir / "derived" / "misconception_identification_example.csv"
)